In [1]:
import pandas as pd

# Getting the results and enrolments data

This section of the notebook involves using the pandas dataframe to import the data from the .csv file.

The data is stored in a root folder titled `student_data`

This data is categorised into subsequent years. For example: 2015, 2016, ...

The `years` array needs to be updated should the new data of the new year is added.

Example of file naming convention for the data files:

* Results from 2015: `results2015.csv`
* Enrolments from 2015: `enrolments2015.csv`


In [2]:
def get_data_frames(data_name, root_folder, years):
    """Returns an array of dataframe of the requested data_name
    
    Keyword arguments:
    data_name -- The name of the data to be imported
    root_folder -- The root folder in which all the csv files are present
    years -- An array of years which is the subfolder inside the root_folder
    """
    data = []
    
    for i in range(len(years)):
        file_name = str(data_name) + str(years[i]) + ".csv"
        path = root_folder + "/" + str(years[i]) + "/" + file_name
        data.append(pd.read_csv(path))
    return data

# Constant Values

The following section contains the constant values that are used while data processing

In [3]:
# Root directory of the data
ROOT_FOLDER = "students_data"
RESULTS = "results"
ENROLMENTS = "enrolments"

## Year list from the sub folders of data

The data for the enrolments and results are stored in subfolder in the years.

Instead of hard coding the years in an array, the following section finds the years by the subfolder name which needs to be in years.

They years are converted into `int` datatype and sorted in `years` list.

In [4]:
# Array of subfolders as per the years
import os

# List the subfolders
sub_folders = os.listdir(ROOT_FOLDER)

years = []

for year in sub_folders:
    try:
        years.append(int(year))
    except ValueError:
        continue

# Sorting the year
years.sort()

print(years)

[2015, 2016, 2017, 2018, 2019]


## Result data

In [6]:
# Reading results files
results_data = []

results_data = get_data_frames(RESULTS, ROOT_FOLDER, years)

In [7]:
# Standardising the columns
results_column_header = ["student_id", "course_code", "unit_cohort", "unit_code", "unit_name", "outcome_date", "teaching_calendar", "grade", "mark"]

for i in range(len(results_data)):
    results_data[i].columns = results_column_header

In [8]:
# Combining the results data

results = pd.concat(results_data, axis=0, sort=False).reset_index(drop=True)

In [9]:
# Size of the data
results.shape

(1277496, 9)

## Enrolment data

In [10]:
# Reading Enrolment files

enrolment_data = []

enrolment_data = get_data_frames(ENROLMENTS, ROOT_FOLDER, years)

In [11]:
# Standardising the columns
enrolments_column_header = ["student_id", "course_code", "student_cohort", "school_name", "course_start_date", "course_attempt_status", "gender", "campus_code", "campus_name", "citizenship", "indigenous_type", "date_of_birth", "discontinued_date", "lapsed_date"]

for i in range(len(enrolment_data)):
    enrolment_data[i].columns = enrolments_column_header

In [12]:
# Combining the enrolment data

enrolments = pd.concat(enrolment_data, axis=0, sort=False).reset_index(drop=True)

In [13]:
# Size of the data
enrolments.shape

(139348, 14)

# Merging Results and Enrolments Data

In [14]:
# Removing the repeated columns
enrolments = enrolments.drop(["course_code", "school_name"], axis=1)

In [17]:
# Combining the tables

final_data = results.join(enrolments.set_index('student_id'), on="student_id")

In [18]:
# Sorting the dataset with student ID

final_data = final_data.sort_values(by=['student_id']).reset_index(drop=True)

In [19]:
# Organising date for outcome_date
final_data['outcome_date'] = pd.to_datetime(final_data.outcome_date)

In [20]:
# Organising date for course_start_date
final_data['course_start_date'] = pd.to_datetime(final_data.course_start_date)

In [21]:
# Organising date for date_of_birth
final_data['date_of_birth'] = pd.to_datetime(final_data.date_of_birth)

In [22]:
final_data.shape

(2107370, 20)

## Duplicate data removal from final data

Run the following section after merging the **results** and **enrolments**.

The following code gets rid of the duplicate values.

In [23]:
final_data = final_data.drop_duplicates().reset_index(drop=True)

In [24]:
final_data.shape

(1960101, 20)

# data cleaning - duplicates

This code will remove further duplicates on the basis of given parameters

**TODO**:

* Identify the column which defines the duplicates
* Filter for the data with those column
* Remove the rows

# Encrypt Student ID

The following code must be run before saving the `final_data`.

The code uses sha1 algorithm to encrypt student ID

In [25]:
import hashlib

students = final_data["student_id"].to_list()

encrypted_id = []

for student in students:
    encrypted_id.append(hashlib.sha1(str(student).encode('ASCII')).hexdigest())

final_data["student_id"] = encrypted_id

# Storing the data

The `final_data` is a pandas dataframe.

Run the following section for storing `final_data` into a `final_data.csv` file.

File path: `students_data/combined_data/final_data.csv`

In [26]:
final_data.to_csv(r'students_data/combined_data/final_data.csv', index=False)

# More information about Final Data

**Note: Running this section is optional. It just provides more insight into the data.**

Following section points out the different attributes obtained from the final dataset.

In [27]:
final_data.nunique()

student_id               83099
course_code               1089
unit_cohort                639
unit_code                 2702
unit_name                 2885
outcome_date             75707
teaching_calendar           51
grade                       34
mark                       101
student_cohort             588
course_start_date         1584
course_attempt_status        6
gender                       4
campus_code                 25
campus_name                 25
citizenship                  6
indigenous_type              5
date_of_birth            11043
discontinued_date         9675
lapsed_date                760
dtype: int64

In [28]:
final_data["gender"].value_counts()

F    1045295
M     784658
X        553
U        105
Name: gender, dtype: int64

In [29]:
final_data["indigenous_type"].value_counts()

NEITHER  ABORIGINAL NOR TORRES STRAIT ISLANDER ORIGIN    1803807
OF ABORIGINAL ORIGIN                                       24320
OF ABORIGINAL AND TORRES STRAIT ISLANDER ORIGIN             1962
OF TORRES STRAIT ISLANDER ORIGIN                             454
NO INFORMATION                                                68
Name: indigenous_type, dtype: int64